# Real-photo $\leftrightarrow$ Monet-style-painting  $\quad$  using Cycle GAN

This is a [Kaggle Challenge](https://www.kaggle.com/c/challenges-in-representation-learning-facial-expression-recognition-challenge). Personal notebook for this challenge is [here](https://drive.google.com/file/d/1Q9X-XbCXz51fByFKYXqg47_JtFzA2WHV/view?usp=sharing)

<br><br>

# Brief review of CycleGAN

The goal of cycleGAN is to learn mapping functions (generators) $G:X \rightarrow Y$ and $F:Y \rightarrow X$ between two domains $X$ and $Y$. Such a setup can also be seen as a special case of “adversarial autoencoders”. To acheive this goal
  - Two discriminators (as the name suggest) $D_X$ and $D_Y$ are introduced to distinguish elements $\{x\}$ in $X$ from translated elements $\{F(y)\}$ and elements $\{y\}$ in $Y$ from translated elements $\{G(x)\}$ respectively. Corresponding adversarial loss is
  

\begin{eqnarray}
\mathcal{L}_{GAN}\left(G,D_Y,X,Y\right) & = & \mathbb{E}_{y\sim p_{data}(y)} \log D_Y(y) + \mathbb{E}_{x\sim p_{data}(x)} \log \left( 1-D_Y(G(x)) \right) \\
& +& \mathbb{E}_{x\sim p_{data}(x)} \log D_X(x) + \mathbb{E}_{y\sim p_{data}(y)} \log \left( 1-D_X(F(y)) \right) \
\end{eqnarray}


<br>

 - GAN loss ensures that the disribution of $G(X)$ is indistinguishable from the distribution of $Y$. However, there are infinitely many such mappings (including e.g. mode collapse). Such a map does not guarantee that an individual
input $x$ and output $y$ are paired up in a meaningful way. The solution is to intorduce **cycle consistency loss** that encourages $F(G(x))\simeq x$ and $G(F(y))\simeq y$


\begin{eqnarray}
\mathcal{L}_{cyc}\left(G,F\right) & = &  \mathbb{E}_{x\sim p_{data}(x)} \lVert F(G(x)) - x \lVert_1 \\
& + & \mathbb{E}_{y\sim p_{data}(y)} \lVert G(F(y)) - y \lVert_1 \\
\end{eqnarray}


In sum, the objective can be represetned by

$$
G^*,F^* = \mathrm{arg}\,\mathrm{min}_{G,F}\,\mathrm{max}_{D_Y, D_Y} \left( \mathcal{L}_{GAN} + \lambda_{cyc} \mathcal{L}_{cyc} \right)
$$


where $\lambda_{cyc}$ controls relative importance of the two objectives

<br>

### Identity loss


The input and output of the the image translation are expected to be similar pixel-to-pixel wise. The identiy regularization encourage such preservation

\begin{eqnarray}
\mathcal{L}_{idt} &=&  \mathbb{E}_{x\sim p_{data}(x)} \lVert F(x)-x \lVert_1 + \mathbb{E}_{y\sim p_{data}(y)} \lVert G(y)-y \lVert_1
\end{eqnarray}

The total loss is
$$
\mathcal{L}_{GAN} + \lambda_{cyc} \left( \mathcal{L}_{cyc} +  \lambda_{idt} \mathcal{L}_{idt} \right)
$$


we follow the choice of weights $\lambda_{cyc}=10$ and $\lambda_{idt}=0.5$ from the cycleGAN paper

<br/><br/>
# Implementation

#### Network Architecture
- *Generator*: 3 convolutions with residuals, 2 of them use fractional stride 1/2. use instance normalization.
- *Discriminator*: use 70x70 PatchGANs

#### Training details
- replace $\mathcal{L}_{GAN}$ by least-square loss:
    - $G$ to minimize $\mathbb{E}_{x\sim p_{data}(x)} \left[ (D(G(x))-1)^2 \right]$, 
    - $D$ to minimize $\mathbb{E}_{y\sim p_{data}(y)} \left[ (D(y)-1)^2\right] + \mathbb{E}_{x\sim p_{data}(x)} \left[ D(G(x))^2\right]$
    
    
- reduce model oscillation based on [this](https://arxiv.org/abs/1612.07828)
    - *self-regularization* term, 
    - local adversarial loss,
    - updating the discriminator using a history of refined images.
    
#### others
- Adam with batch size 1
- lr = 0.0003 for the first 300 epochs then decay 0.0002 the next epohs


<br><br>

# Result

#### Training history
- GAN_A : Monet -> Photo  
- GAN_B : Photo -> Monet

<img src="train_hist900.png" width="1400"/>

## Monet to Photo over training epochs (100 ,400, 900 epochs)

<img src="monet2photo.png" width="1400"/>

## Photo to Monet over training epochs (100 ,400, 900 epochs)

<img src="photo2monet.png" width="1400"/>